In [ ]:
from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
import lxml
import requests
import re
import pandas as pd
import numpy as np
import csv
from lxml import html
import requests
from langdetect import detect

### STEP 1

#### STEP 1.1 -> Getting all the URLS

In [ ]:
# Needs to be changed to 201 to 301
outfile = open("test.txt", "w")
for i in range(201,301):
    
    base_url = "https://www.goodreads.com/list/show/1.Best_Books_Ever?page="+str(i)

    html = urlopen(base_url)
    soup = BeautifulSoup(html.read(), features="lxml")

    links = []
    regex = "/book/show/."

    for link in soup.find_all("a"):
        if re.match(regex, str(link.get("href"))):
            links.append("https://www.goodreads.com"+link.get("href"))
    
    #issues of duplicates        
    links = list(dict.fromkeys(links))
    
    for link in links:
        outfile.write(link+"\n")
outfile.close()

#### STEP 1.2 -> getting the HTML from the URLs, getting these HTMLs in different files

do not run the code below, this is just to show

In [ ]:
from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
import lxml
import requests
import re
import os

# First, I make sure i am in the directory that has the text file with the URLS
path = "C:/Users/Guillaume/Desktop/ADM_HM3"
os.chdir(path)

i = 1
page = 1

with open("test_urls.txt", "r") as urls:
    # I previously created the 100 directories, from page 201 to page 300
    # I change the working directory for which i want the HTMLs 
    path = "C:/Users/Guillaume/Desktop/ADM_HM3/page_201"
    os.chdir(path)

    for url in urls.readlines()[0:]:

        html = urlopen(url)

        soup = BeautifulSoup(html.read(), 'html.parser')
        
        # The logig here is not the best but it works- 
        # Basically, once every 100 urls treated I chance the directory
        
        if (i%101) != 0:
                # We stay in the current directory
            with open("article_"+str(i)+".html", "w", encoding="utf-8") as outfile:
                outfile.write(str(soup))

            i = i + 1

        else :

            page = page + 1
            path = "C:/Users/Guillaume/Desktop/ADM_HM3/page_20"+str(page)
            os.chdir(path)

            with open("article_"+str(i)+".html", "w", encoding="utf-8") as outfile:
                outfile.write(str(soup))

This has been done locally

#### STEP 1.3 -> Parsing the HTMLs files to retrieve all the required info

In [ ]:
# All functions to retrieve the informations

def get_title(soup):
    title = soup.find(id="bookTitle").contents[0].strip()
    return title

def get_series(soup):
    serie = soup.find(id="bookSeries").get_text().strip().replace("(","").replace(")","")
    regex = "#.*"
    serie = re.sub(regex, "",serie)
    return serie

def get_author(soup):
    author = soup.find_all(class_="authorName__container")
    author_list = " ".join([x.get_text().strip() for x in author])
    return author_list

def get_pages(soup):
    pages = soup.find(itemprop ="numberOfPages").get_text().split()[0]
    return int(pages)

def get_ratingValue(soup):
    ratingValue = soup.find(itemprop ="ratingValue").get_text().split()[0]
    return float(ratingValue)

def get_ratingCount(soup):
    test = []
    ratingCount = soup.find_all(class_ ="gr-hyperlink", href="#other_reviews")
    for x in ratingCount:
        test.append(x.get_text().split())
    return int(test[0][0].replace(",",""))
    
def get_reviewCount(soup):
    test = []
    ratingCount = soup.find_all(class_ ="gr-hyperlink", href="#other_reviews")
    for x in ratingCount:
        test.append(x.get_text().split())
    return int(test[1][0].replace(",",""))

def get_date(soup):
    regex = "by.*"
    date = soup.find_all(class_ = "row")
    date = re.sub(regex,"" ,str(date[1].contents[0]).replace("Published","")).strip()
    return date

def get_setting(soup):
    string = re.compile("/places/.")
    test = soup.find_all(href=string)
    
    places_list = " ".join([(str(a.contents[0])) for a in test])

    return places_list

def get_plot(soup):
    plot = soup.find(id="description").get_text().strip()
    return plot 

def get_char(soup):
    string = re.compile("/characters/.")
    test = soup.find_all(href=string)

    char_list= " ".join([(str(a.contents[0])) for a in test])

    return char_list

def get_url(soup):

    regex = "https://www.goodreads.com/book/show/."
    ratingCount = soup.find(href =re.compile(regex), rel="canonical")
    return ratingCount.get('href')
    

In [177]:
# We want 1 TSV file for 1 article
# In order to do this -> we first loop over the HTML
# We check if the language is indeed english
# for each HTML we create a new TSV and input the required DATA

i = 4

while i <= 4:
    html = open("article_"+str(i)+".html", "r", encoding="utf-8")
    soup = BeautifulSoup(html.read(), 'html.parser')
        
    with open("html_tsv_test_"+str(i)+".tsv", 'w', encoding="utf-8") as f_output:
        
        # We check the language of the splot
        if detect(get_plot(soup)) == "en":
            
            test_list_tsv = [get_title(soup), 
                             get_series(soup), 
                             get_author(soup),
                             get_ratingValue(soup),
                             get_ratingCount(soup), 
                             get_reviewCount(soup),
                             get_plot(soup),
                             get_pages(soup),
                             get_date(soup),
                             get_setting(soup),
                             get_char(soup),
                             get_url(soup)]
            
            header_list = ["bookTitle","bookSeries","bookAuthors","ratingValue","ratingCount","reviewCount","Plot","NumberofPages","Date","Characters","Setting","URL"]

            tsv_output = csv.writer(f_output, delimiter='\t')
            tsv_output.writerow(header_list)
            tsv_output.writerow(test_list_tsv)
            
            i = i + 1


## Step 2.0

Preprocessing
- Removing stopwords
- Removing punctuation
- Stemming
- Anything else you think it's needed ?

What is "anything else"?

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

In [178]:
# We do the preprocessing and the creation of the vocabulary in one go
vocabulary = []

# start looping over the tsv files
for x in range(1,5):

    data = pd.read_csv("html_tsv_test_"+str(x)+".tsv", sep = "\t", encoding = "utf-8")
    plot = data['Plot'].item()
    
    #tokenisation
    word_tokens = word_tokenize(plot) 
    # stopwords
    stop_words = set(stopwords.words('english'))
    filter_stopsword = [w for w in word_tokens if not w in stop_words]
    # punctuations
    filter_punc = [w for w in filter_stopsword if w.isalnum()]
    # stemming
    ps = PorterStemmer() 
    filter_stem = [ps.stem(w) for w in filter_punc]
    # single string
    clean_plot = " ".join(filter_stem)
    
    # THIS PART TO REPLACE THE OLD PLOT
    # replace the old plot with the new one
    data["Plot"] = data["Plot"].replace(data['Plot'].item(),clean_plot)
    # ideally we will save over the old file
    data.to_csv("article_"+str(x)+".tsv", sep = '\t')
    
    # THIS PART TO CREATE THE VOCABULARY
    vocabulary.append(filter_stem)
    
vocabulary

[['A', 'power', 'novel', 'spiritu', 'bond', 'man', 'anim'],
 ['there',
  'door',
  'end',
  'silent',
  'corridor',
  'and',
  'haunt',
  'harri',
  'pottter',
  'dream',
  'whi',
  'els',
  'would',
  'wake',
  'middl',
  'night',
  'scream',
  'terror',
  'harri',
  'lot',
  'mind',
  'fifth',
  'year',
  'hogwart',
  'defens',
  'against',
  'dark',
  'art',
  'teacher',
  'person',
  'like',
  'poison',
  'honey',
  'big',
  'surpris',
  'gryffindor',
  'quidditch',
  'team',
  'loomi',
  'there',
  'door',
  'end',
  'silent',
  'corridor',
  'and',
  'haunt',
  'harri',
  'pottter',
  'dream',
  'whi',
  'els',
  'would',
  'wake',
  'middl',
  'night',
  'scream',
  'terror',
  'harri',
  'lot',
  'mind',
  'fifth',
  'year',
  'hogwart',
  'defens',
  'against',
  'dark',
  'art',
  'teacher',
  'person',
  'like',
  'poison',
  'honey',
  'big',
  'surpris',
  'gryffindor',
  'quidditch',
  'team',
  'loom',
  'terror',
  'ordinari',
  'wizard',
  'level',
  'exam',
  'but',
 

In [218]:
# the Vocabulary is a list of list, we flatten that shit
flat_vocabulary = [item for sublist in vocabulary for item in sublist]
# putting the elements in a dico will eliminate the duplicates
dico_voc = dict.fromkeys(flat_vocabulary)

# We get the "integers" are asked to do
i = 1
for k,v in dico_voc.items():
    dico_voc[k] = i
    i = i +1

print(dico_voc)

{'A': 1, 'power': 2, 'novel': 3, 'spiritu': 4, 'bond': 5, 'man': 6, 'anim': 7, 'there': 8, 'door': 9, 'end': 10, 'silent': 11, 'corridor': 12, 'and': 13, 'haunt': 14, 'harri': 15, 'pottter': 16, 'dream': 17, 'whi': 18, 'els': 19, 'would': 20, 'wake': 21, 'middl': 22, 'night': 23, 'scream': 24, 'terror': 25, 'lot': 26, 'mind': 27, 'fifth': 28, 'year': 29, 'hogwart': 30, 'defens': 31, 'against': 32, 'dark': 33, 'art': 34, 'teacher': 35, 'person': 36, 'like': 37, 'poison': 38, 'honey': 39, 'big': 40, 'surpris': 41, 'gryffindor': 42, 'quidditch': 43, 'team': 44, 'loomi': 45, 'loom': 46, 'ordinari': 47, 'wizard': 48, 'level': 49, 'exam': 50, 'but': 51, 'thing': 52, 'pale': 53, 'next': 54, 'grow': 55, 'threat': 56, 'neither': 57, 'magic': 58, 'govern': 59, 'author': 60, 'grasp': 61, 'tighten': 62, 'must': 63, 'discov': 64, 'true': 65, 'depth': 66, 'strength': 67, 'friend': 68, 'import': 69, 'boundless': 70, 'loyalti': 71, 'shock': 72, 'price': 73, 'unbear': 74, 'fate': 75, 'depend': 76, 'cou

### 2.1. Conjunctive query

#### 2.1.1 Create your index

In order to create this index we must loop for EACH word in ALLLLLLLL the tsv files we have. This will take time and undoubtedly crash a few times. 

We start this horrible looping.
For each word in the vocabulary -> we loop through each plot -> if the word is in that plot we retrieve we get the "document_id"
"where document_i is the id of a document that contains the word."

now...do documents have ids? -> We MUST align the article_id.tsv with the document_id in this dico

In [221]:
index = {}

for k,v in dico_voc.items():
    for x in range(1,5):
        with open("article_"+str(x)+".tsv", "r") as article:
            data = pd.read_csv(article, sep = "\t", encoding = "utf-8")
            if k in data["Plot"].item():
                if k not in index:
                    # this would require to be adjusted if the id is just an integer
                    index[k] = ["article_"+str(x)]
                elif k in index:
                    index[k].append("article_"+str(x))
print(index)

{'A': ['article_1'], 'power': ['article_1'], 'novel': ['article_1'], 'spiritu': ['article_1'], 'bond': ['article_1'], 'man': ['article_1', 'article_3'], 'anim': ['article_1'], 'there': ['article_2'], 'door': ['article_2'], 'end': ['article_2', 'article_3'], 'silent': ['article_2'], 'corridor': ['article_2'], 'and': ['article_2'], 'haunt': ['article_2'], 'harri': ['article_2', 'article_4'], 'pottter': ['article_2'], 'dream': ['article_2', 'article_4'], 'whi': ['article_2'], 'els': ['article_2'], 'would': ['article_2'], 'wake': ['article_2'], 'middl': ['article_2'], 'night': ['article_2'], 'scream': ['article_2'], 'terror': ['article_2'], 'lot': ['article_2'], 'mind': ['article_2'], 'fifth': ['article_2'], 'year': ['article_2', 'article_4'], 'hogwart': ['article_2', 'article_4'], 'defens': ['article_2'], 'against': ['article_2'], 'dark': ['article_2'], 'art': ['article_2', 'article_3', 'article_4'], 'teacher': ['article_2'], 'person': ['article_2'], 'like': ['article_2'], 'poison': ['art

In [204]:
# Easier example for the one above. in case the logic needs to be re-examined
index_test = {"a":[11]}

voc = {"a":1, "b":2, "d":3}

for k,v in voc.items():
    if k not in index_test:
        index_test[k] = v
    else:
        index_test[k].append(v)

print(index_test)

{'a': [11, 1], 'b': 2, 'd': 3}


#### 2.1.2 Execute the query

In [256]:
x = input()
split_input = x.split()

# my guess is that we should also do the pre-processing here. otherwise we if we ask for "harry" we get no response
# that is because "harry" transformed into "harri" following the pre-processing. 

sets = []
for y in split_input:
    if y in index:
        sets.append(index[y])


# What we want is the intersection 

power


In [257]:
print(sets)

[['article_1']]
